In [61]:
from kaggle.api.kaggle_api_extended import KaggleApi
import csv
import os

In [62]:
class KaggleLoadSubmit:
    def __init__(self, competition_name, csv_dir):
        """Kaggleのデータのダウンロードおよび提出を行うクラス

        Args:
            competition_name (string): Kaggleのコンペ名
            csv_dir (string): ダウンロード先と、提出先のcsvのディレクトリをstringで指定
        """
        self.kaggle_api = KaggleApi()
        self.csv_dir = csv_dir
        self.competition_name = competition_name
        if self.csv_dir[-1] != '\\':
            self.csv_dir += '\\'
        self.kaggle_api.authenticate()

    def download_zip(self):
        """コンペティションのzipファイルをダウンロードする
        """
        ret = self.kaggle_api.competition_download_files(self.competition_name, self.csv_dir, force=True)
        print(ret)

    def submit_csv(self, csv_name, message):
        """csvデータを提出する

        Args:
            csv_name (string): csvの名前
            message (string): 提出メッセージ
        """
        csv_path = self.csv_dir + csv_name
        self.kaggle_api.competition_submit(csv_path, message, self.competition_name)

In [63]:
kls = KaggleLoadSubmit('titanic', 'C:\\Users\\Takanori\\Desktop\\Kaggle\\titanic\\data')

# Titanic
https://www.kaggle.com/c/titanic/

In [64]:
import numpy as np
import pandas as pd

In [65]:
# データの読み込み
data_dir = 'C:\\Users\\Takanori\\Desktop\\Kaggle\\titanic\\data'
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

def fix(data):
    # 明らかに要らないデータを削除
    delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
    data.drop(delete_columns, axis=1, inplace=True)
    # 文字列を数値に
    data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
    data['Embarked'].fillna(('S'), inplace=True)
    data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
    data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
    data['IsAlone'] = 0
    data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

for data in [train, test]:
    fix(data)


In [66]:
# LightGBM
# 1.データを学習用と検証用に分割
X_train = train.drop('Survived', axis=1)
Y_train = train['Survived']
X_test = test[:]

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0, stratify=Y_train)

In [67]:
# 2. カテゴリ変数（文字列等）を列挙
categorical_features = ['Embarked', 'Pclass', 'Sex']

In [68]:
# 3.predict
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, Y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, Y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {'objective': 'binary'}

model = lgb.train(params, lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=10,
                  num_boost_round=1000,
                  early_stopping_rounds=10)

Y_pred = model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000155 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 205
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

C:\Users\Takanori\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Takanori\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Takanori\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Takanori\

In [69]:
Y_pred = (Y_pred > 0.5).astype(int)

In [70]:
# save_csv
sub = pd.read_csv(os.path.join(data_dir, 'gender_submission.csv'))
sub['Survived'] = Y_pred
sub.to_csv(os.path.join(data_dir, 'my_submission.csv'), index=False)

In [71]:
kls.submit_csv(os.path.join('my_submission.csv'), '0316')

100%|██████████| 3.18k/3.18k [00:04<00:00, 796B/s]
